In [3]:
import pandas as pd
import json
import xml.etree.ElementTree as ET # Импортируем модуль ElementTree

# ИЗВЛЕКАЕМ КОНТЕНТ ИЗ XML-ФАЙЛА

In [4]:
tree = ET.parse('data/menu.xml')

### КОРЕНЬ
Запишем в переменную root корневой узел дерева tree и посмотрим, как выглядит содержимое переменной root, для чего выполним код:

In [5]:
root = tree.getroot()
display(root)

<Element 'menu' at 0x000001685BD70C20>

In [6]:
display(type(root))

xml.etree.ElementTree.Element

### ПОТОМКИ

Для того чтобы посмотреть список потомков корневого узла, выполним следующий код:

In [9]:
display(list(root))

[<Element 'dish' at 0x000001685BD70BD0>,
 <Element 'dish' at 0x000001685BD70A40>]

Если у узла нет потомков, то вернётся пустой список — [].

In [10]:
# Сколько потомков у элемента root?
display(len(list(root)))

2

Итак, использование list(root) возвращает список потомков указанного узла. У узла root, который представляет меню, два потомка, а именно — два блюда, которые представлены тегами dish.

Для того чтобы получить список потомков второго блюда в нашем меню и вывести его на экран, выполним код:

In [11]:
 display(list(root[1]))

[<Element 'price' at 0x000001685BD709F0>,
 <Element 'weight' at 0x000001685BD709A0>,
 <Element 'class' at 0x000001685BD70720>]

### АТРИБУТЫ И ТЕГИ

Как было сказано ранее, у узлов могут быть параметры, или атрибуты. Например, у узлов dish есть атрибут name, который хранит название блюда.

Мы можем непосредственно обратиться к атрибутам, используя attrib.

Выведем на экран атрибуты первого блюда из меню:

In [12]:
 display(root[0].attrib)

{'name': 'Кура'}

В XML-узлах часто хранятся количественные показатели. Эти показатели хранятся в виде текста, и прочитать их можно, обратившись к атрибуту text у соответствующего объекта типа ElementTree.Element.

Например, возьмём узел price первого блюда из меню:

In [13]:
display(root[0][0])

<Element 'price' at 0x000001685BD70B80>

Теперь прочитаем значение этого узла с помощью text:

In [14]:
display(root[0][0].text)

'40'

**Все значения в XML, даже числовые, хранятся как строки, поэтому преобразовывать их к нужному типу вам нужно самим.**

Если вы хотите прочитать наименование тега конкретного узла, необходимо использовать tag. Например, получим наименование тега корневого узла:

In [15]:
display(root.tag)

'menu'

In [16]:
# Какое наименование имеет тег узла root[0][2]?
display(root[0][2])


<Element 'class' at 0x000001685BD70A90>

### <center> ИСПОЛЬЗОВАНИЕ ЦИКЛОВ

На этом шаге мы решим задачу вывода на экран наименование всех блюд из меню, а также информацию о них (иными словами, нам необходимо обойти дерево и вывести на экран значения его листьев).

Используя цикл for, автоматизируем обход дерева. Для этого напишем следующий код:

In [17]:
for dish in root:
    for param in dish:
        print(dish.attrib['name'], param.tag, param.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



В этом коде реализован следующий алгоритм:

В первом (внешнем) цикле перебираем потомков корня дерева (root). Потомки перебираются последовательно при помощи переменной dish. Это отдельные блюда из меню.
Во втором (вложенном) цикле аналогичным образом перебираем потомков каждого блюда. Этими потомками являются параметры блюда — его цена (price), вес (weight) и класс (class).
После этого выводим на экран название блюда (значение атрибута name), название очередного параметра (tag) и его значение (text).
Дополнительная функция print() в цикле верхнего уровня предназначена для организации более удобного восприятия информации — между отдельными блюдами будет выведена пустая строка.

# <center> 10. XML. Загружаем, создаем, сохраняем

Cтандартных средств для превращения XML-файла в DataFrame нет, но если XML-файл содержит чёткую структуру, сделать это средствами Python достаточно просто.

Покажем, как это сделать, на примере работы с нашим файлом-меню.

✍ Реализуем следующий алгоритм:

1. Загрузить XML-структуру файла menu.xml в переменную root.
2. Создать пустой список df_list (в него будем добавлять строчки итоговой таблицы).
З. Заранее создать список column_names с именами столбцов — название блюда (name), его цена (price), вес (weight) и класс (class).
4. В цикле организовать обход xml-дерева из корня по всем потомкам.
5. На каждой итерации цикла сформировать в виде списка строку таблицы, содержащую информацию: наименование блюда (атрибут name узла dish) и значения потомков этого узла — узлов price, weight, class.
6. Добавить сформированную строку в список df_list, используя метод append().
7. Сформировать из вложенного списка DataFrame. Имена для столбцов взять из списка column_names.

In [19]:
tree = ET.parse('data/menu.xml')
root = tree.getroot()

import pandas as pd
column_names = ['name', 'price', 'weight', 'class']
df_list = []

for dish in root:
    row = [dish.attrib['name'], dish[0].text, dish[1].text, dish[2].text]
    df_list.append(row)
    df = pd.DataFrame(df_list, columns=column_names)
display(df)

,name,price,weight,class
0,Кура,40,300,Мясо
1,Греча,20,200,Крупа


### <center> СОЗДАЁМ XML-ФАЙЛ

Воссоздадим структуру нашего исходного XML-файла с нуля,  руководствуясь общими рекомендациями.

Чтобы создать корень дерева, используем метод Element() из класса ElementTree:

In [20]:
new_root = ET.Element('menu')
display(new_root)

<Element 'menu' at 0x000001685BE7C4A0>

Теперь мы можем добавлять новые узлы в наше дерево, используя метод SubElement() из того же класса.

Добавим в наше меню двух потомков корневого узла, которые будут представлять два блюда, то есть будут узлами dish:

In [21]:
dish1 = ET.SubElement(new_root, 'dish', name='Кура')
dish2 = ET.SubElement(new_root, 'dish', name='Греча')
display(list(new_root))

[<Element 'dish' at 0x000001685BDAE2C0>,
 <Element 'dish' at 0x000001685BDAE770>]

В метод SubElement() мы передали первым аргументом узел, к которому добавляем потомка, вторым аргументом — наименование нового тега (dish),  третьим аргументом — наименование атрибута нового узла( name ) и его значение.

***Аналогичным образом можно добавлять новые узлы к любым существующим узлам, не только к корню.***

Добавим в создаваемую структуру по три потомка (атрибута) к двум новым узлам, которые будут содержать информацию о блюде — о его цене (price), весе (weight) и классе (class), а также значение этих атрибутов:

In [22]:
price1 = ET.SubElement(dish1, "price").text = "40"
weight1 = ET.SubElement(dish1, "weight").text = "300"
class1 = ET.SubElement(dish1, "class").text = "Мясо"
display(list(dish1))

price2 = ET.SubElement(dish2, "price").text = "20"
weight2 = ET.SubElement(dish2, "weight").text = "200"
class2 = ET.SubElement(dish2, "class").text = "Крупа"
display(list(dish2))

[<Element 'price' at 0x000001685BDAE9A0>,
 <Element 'weight' at 0x000001685BDAC220>,
 <Element 'class' at 0x000001685D00E630>]

[<Element 'price' at 0x000001685BDAC3B0>,
 <Element 'weight' at 0x000001685D00E6D0>,
 <Element 'class' at 0x000001685D00E860>]

Проверим визуально корректность созданной нами структуры, выполнив фрагмент кода, разработанного ранее:

In [23]:
for dish in new_root:
    for param in dish:
        print(dish.attrib['name'], param.tag, param.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



### <center> СОХРАНЕНИЕ XML-ФАЙЛА

Преобразуем созданный нами объект типа ElementTree.Element в строку c помощью метода ***tostring()***, передав наше новое дерево как аргумент. Сохраним эту строку на диске, используя стандартные средства Python::

In [24]:
new_root_string = ET.tostring(new_root)
with open("new_menu.xml", "wb") as f:
    f.write(new_root_string)

✍️ Файл записан! Откройте его и посмотрите, что получилось.

Возможно, вы увидите проблему, связанную с кодировкой. Что делать в этом случае? Как вариант — записать файл, используя сам класс ElementTree() :

In [25]:
ET.ElementTree(new_root).write('new_menu_good.xml', encoding="utf-8")

Для этого мы передаём в класс ElementTree() наше дерево (не его строковое представление) и вызываем метод write(). В метод мы передаём путь к новому файлу и нужную нам кодировку.

Python- 16.ИТОГИ. Там все ссылки.
ZIP

[Working with zip files in Python](https://www.geeksforgeeks.org/working-zip-files-python/)

EXCEL

[Автоматизация Excel с помощью Python](https://medium.com/nastia-shu/%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%B5-%D0%BD%D0%B5-%D0%BD%D1%83%D0%B6%D0%BD%D0%BE-%D0%BE%D1%82%D0%BA%D1%80%D1%8B%D0%B2%D0%B0%D1%82%D1%8C-%D1%81%D0%BE%D1%82%D0%BD%D0%B8-%D1%84%D0%B0%D0%B9%D0%BB%D0%BE%D0%B2-%D0%B2-excel-e0a1f5a9e9a7)

[Использование Python и Excel для обработки и анализа данных](https://habr.com/ru/company/otus/blog/331998/)

[How to Work with Excel files in Pandas](https://towardsdatascience.com/how-to-work-with-excel-files-in-pandas-c584abb67bfb)

[Pandas read_excel() – Reading Excel File in Python](https://www.journaldev.com/33306/pandas-read_excel-reading-excel-file-in-python)

[Python Excel Tutorial: The Definitive Guide](https://www.datacamp.com/community/tutorials/python-excel-tutorial)

[Tutorial Using Excel with Python and Pandas](https://www.dataquest.io/blog/excel-and-pandas/)

JSON

[Парсинг JSON](https://all-python.ru/osnovy/json.html)
[Working With JSON Data in Python](https://realpython.com/python-json/)
[Python JSON ](https://www.programiz.com/python-programming/json)
XML

[Работа с XML из Python](https://codecamp.ru/blog/python-manipulating-xml/)
[Processing XML in Python — ElementTree](https://towardsdatascience.com/processing-xml-in-python-elementtree-c8992941efd2)


